In [1]:
# my_list = [
#     "idr0001", "idr0007", "idr0012", "idr0025", "idr0037", "idr0081", "idr0094", "idr0129", "nidr0001", "nidr0006", "nidr0011", "nidr0016", "nidr0021", "nidr0026", "nidr0031",
#     "idr0002", "idr0008", "idr0013", "idr0028", "idr0056", "idr0086", "idr0115", "idr0130", "nidr0002", "nidr0007", "nidr0012", "nidr0017", "nidr0022", "nidr0027", "nidr0032",
#     "idr0003", "idr0009", "idr0017", "idr0030", "idr0069", "idr0088", "idr0120", "idr0133", "nidr0003", "nidr0008", "nidr0013", "nidr0018", "nidr0023", "nidr0028",
#     "idr0005", "idr0010", "idr0020", "idr0033", "idr0072", "idr0089", "idr0123", "idr0140", "nidr0004", "nidr0009", "nidr0014", "nidr0019", "nidr0024", "nidr0029",
#     "idr0006", "idr0011", "idr0022", "idr0035", "idr0080", "idr0093", "idr0128", "idr0145", "nidr0005", "nidr0010", "nidr0015", "nidr0020", "nidr0025", "nidr0030"
# ]


# for idr in sorted(my_list):
#     print(f"./generate_tile_hists.py -d /mnt/cephfs/mir/jcaicedo/morphem/dataset/images/{idr}/ -o ~/jpeters/histograms/")

In [2]:
import polars as pl
import json
from collections import defaultdict
csv = pl.read_csv('./thresholds.csv', has_header=True) 
csv.head()

study,plate,keywords,keyword_type,value_type,threshold,direction,threshold_type
str,str,str,str,str,i64,str,str
"""idr1""","""all""",null,null,"""pc""",10,"""top""","""%"""
"""idr2""","""all""",null,null,"""live""",15,"""top""","""%"""
"""idr3""","""all""",null,null,"""pc""",20,"""top""","""%"""
"""idr5""","""all""",null,null,"""pc""",20,"""top""","""%"""
"""idr6""","""all""",null,null,"""live""",10,"""top""","""%"""


In [3]:

as_json = defaultdict(dict)
for study, plate, keywords, keyword_type, value_type, thresh, direction, thresh_type in csv.iter_rows():
    study_split = study.split('r')
    places_to_add = 4 - len(study_split[1])
    study_correct = study_split[0] + "r" + "0"*places_to_add + study_split[1]

    if not keywords:
        keywords = "default"
    
    if plate not in as_json[study_correct]: as_json[study_correct][plate] = {}
    
    as_json[study_correct][plate][keywords] = {
                "keyword_type": keyword_type,
                "value_type": value_type,
                "threshold": int(thresh),
                "threshold_type": thresh_type,
                "direction": direction
    }    

with open('./thresholds.json', 'w') as f:
    f.write(json.dumps(as_json, indent=2))

In [4]:
with open('./thresholds.json', 'r') as f:
    json_loaded = json.loads(f.read())